In [11]:
!pip install --upgrade langchain langchain-core langchain-openai openai pydantic rich gradio


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 64.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 35.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 62.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 62.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 65.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 46.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.7/815.7 kB 43.6 MB/s  0:00:00
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 63.8 MB/s  0:00:00
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━

In [5]:
pip install gradio --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 37.9 MB/s  0:00:01m0:00:0100:01
  Attempting uninstall: gradio
    Found existing installation: gradio 5.43.1
    Uninstalling gradio-5.43.1:
      Successfully uninstalled gradio-5.43.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install --upgrade langchain



In [8]:
!pip install -q gradio langchain-openai


In [5]:
import os
import json
import gradio as gr
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# File for storing favorite recipes
FAVORITES_FILE = "favorite_recipes_openai.json"

print("✅ Setup complete.")

✅ Setup complete.


In [6]:
# --- Functions for Favorites ---

def load_favorites():
    """Loads favorite recipes from the JSON file."""
    if os.path.exists(FAVORITES_FILE):
        with open(FAVORITES_FILE, "r") as f:
            return json.load(f)
    return {}

def save_favorites(favorites):
    """Saves favorites to the JSON file."""
    with open(FAVORITES_FILE, "w") as f:
        json.dump(favorites, f, indent=4)

def add_favorite(recipe_name, recipe_text):
    """Adds a recipe to favorites."""
    if not recipe_name or not recipe_text:
        return "⚠️ Please generate a recipe first."
    
    favorites = load_favorites()
    favorites[recipe_name] = recipe_text
    save_favorites(favorites)
    return f"🌟 '{recipe_name}' has been saved to your favorites!"

def get_favorite_names():
    """Returns all favorite recipe names."""
    favorites = load_favorites()
    return list(favorites.keys())

def get_favorite_recipe(recipe_name):
    """Gets a specific favorite recipe."""
    favorites = load_favorites()
    return favorites.get(recipe_name, "❌ Recipe not found.")


In [7]:
# --- Prompt Template ---
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a world-class chef and culinary expert. 
            Generate a recipe based ONLY on the ingredients the user provides. 
            
            Format the response with these sections:
            
            ## 🍽️ Recipe Name
            (A creative and enticing name)
            
            ## ✨ Description
            (Mouth-watering description)
            
            ## 🥕 Ingredients
            - List all necessary ingredients (include user ingredients + common extras if needed)
            
            ## 👨‍🍳 Instructions
            1. Step-by-step clear instructions
            2. Easy to follow for home cooks
            
            ## ⏱️ Prep Time
            (Estimated time)
            
            ## 🍳 Cook Time
            (Estimated time)
            
            ---
            🍽️ Have a good meal!
            
            Keep output in Markdown format. Do not include any text outside of this format.
            """
        ),
        ("human", "Here are the ingredients: {ingredients}. Create a recipe."),
    ]
)

# --- AI Model ---
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# --- Chain ---
recipe_chain = prompt_template | model | StrOutputParser()

def generate_recipe(ingredients):
    """Generate a recipe given user ingredients."""
    try:
        for chunk in recipe_chain.stream({"ingredients": ingredients}):
            yield chunk
    except Exception as e:
        yield f"❌ Error: {e}. Please check your API key."


In [8]:
# --- Gradio UI ---
with gr.Blocks(title="AI Recipe Assistant with OpenAI") as demo:
    gr.Markdown(
        """
        # 👩‍🍳 AI Recipe Assistant  
        Enter the **ingredients** you have, and our AI Chef will create a delicious recipe for you!  
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            ingredients_input = gr.Textbox(
                label="Your Ingredients", 
                placeholder="e.g., chicken, garlic, basil, chili", 
                lines=2,
                container=True
            )
            generate_button = gr.Button("🍲 Generate Recipe", variant="primary")
            
        with gr.Column(scale=1):
            save_name_input = gr.Textbox(
                label="Save Recipe As", 
                placeholder="e.g., My Basil Chicken", 
                lines=1,
                container=True
            )
            save_button = gr.Button("💾 Save to Favorites")

    recipe_output = gr.Markdown(label="Generated Recipe")

    gr.Markdown("---")
    gr.Markdown("### 🌟 Your Favorite Recipes")

    with gr.Row():
        with gr.Column():
            favorite_dropdown = gr.Dropdown(
                label="Select a Favorite Recipe", 
                choices=get_favorite_names()
            )
            load_favorite_button = gr.Button("📖 Load Favorite", variant="secondary")

    favorite_output = gr.Markdown(label="Favorite Recipe Details")

    status_message = gr.Textbox(
        label="Status", 
        value="", 
        interactive=False, 
        container=False
    )

    # --- Button Actions ---
    generate_button.click(
        fn=generate_recipe, 
        inputs=ingredients_input, 
        outputs=recipe_output
    )

    save_button.click(
        fn=add_favorite, 
        inputs=[save_name_input, recipe_output], 
        outputs=status_message
    )

    load_favorite_button.click(
        fn=get_favorite_recipe, 
        inputs=favorite_dropdown, 
        outputs=favorite_output
    )
    
# --- Launch App ---
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://c0cb3c24ca59f458f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
